In [ ]:
! pip install datasets
! pip install pycaret


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 7.5 MB/s eta 0:00:00
 

In [ ]:
from datasets import load_dataset
import gensim.downloader as api
import numpy as np

# Check if GPU is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the GloVe model
glove_model = api.load("glove-wiki-gigaword-100")  # Use the GloVe embeddings (100-dimensional)

[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
# def get_sentence_vector(sentence):
#     words = sentence.split()
#     word_vectors = [glove_model[word] for word in words if word in glove_model]
#     return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(glove_model.vector_size)
dataset = load_dataset("PiC/phrase_similarity")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.47k [00:00<?, ?B/s]

phrase_similarity.py:   0%|          | 0.00/4.75k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

PS-hard/validation/0000.parquet:   0%|          | 0.00/202k [00:00<?, ?B/s]

PS-hard/test/0000.parquet:   0%|          | 0.00/403k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7004 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['phrase1', 'phrase2', 'sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 7004
    })
    validation: Dataset({
        features: ['phrase1', 'phrase2', 'sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['phrase1', 'phrase2', 'sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 2000
    })
})

In [ ]:
# Apply mapping with updated function
for split in ['train', 'validation', 'test']:
    dataset[split] = dataset[split].map(
        lambda x: {
            'sent': x['phrase1']+' '+x['phrase2']
        }
    )

Map:   0%|          | 0/7004 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
dataset["train"]["sent"][0]

'newly formed camp recently made encampment'

In [1]:
np.concatenate([[1,2], [3,4]])

NameError: name 'np' is not defined

In [ ]:
# Ensure get_sentence_vector outputs lists of float32
def get_sentence_vector(sentence):
    words = sentence.split()
    word_vectors = [glove_model[word] for word in words if word in glove_model]
    avg_vector = np.mean(word_vectors, axis=0) if word_vectors else np.zeros(glove_model.vector_size)
    return avg_vector.astype(np.float32).tolist()  # Convert to list of float32

# Apply mapping with updated function
for split in ['train', 'validation', 'test']:
    dataset[split] = dataset[split].map(
        lambda x: {
            'ph1Vec': get_sentence_vector(x['sent'])
            # 'ph2Vec': get_sentence_vector(x['phrase2'])
        }
    )

# # Apply mapping with updated function
# for split in ['train', 'validation', 'test']:
#     dataset[split] = dataset[split].map(
#         lambda x: {
#             'Vec': ((np.array(x['ph1Vec']) + np.array(x['ph2Vec'])) / 2).tolist()

#         }
#     )


# # Apply mapping with updated function
# for split in ['train', 'validation', 'test']:
#     dataset[split] = dataset[split].map(
#         lambda x: {


#             'hVec': np.hstack([x['ph1Vec'],x['ph2Vec']]).tolist()

#         }
#     )

Map:   0%|          | 0/7004 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['phrase1', 'phrase2', 'sentence1', 'sentence2', 'label', 'idx', 'sent', 'ph1Vec'],
        num_rows: 7004
    })
    validation: Dataset({
        features: ['phrase1', 'phrase2', 'sentence1', 'sentence2', 'label', 'idx', 'sent', 'ph1Vec'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['phrase1', 'phrase2', 'sentence1', 'sentence2', 'label', 'idx', 'sent', 'ph1Vec'],
        num_rows: 2000
    })
})

In [ ]:
# ! pip install "cuml-cu12==23.06" -f https://rapids.ai/wheels/stable


# ! pip3 install auto-sklearn

In [ ]:
X_train = np.array(dataset['train']['ph1Vec'])
y_train = np.array(dataset['train']['label'])
X_val = np.array(dataset['validation']['ph1Vec'])
y_val = np.array(dataset['validation']['label'])

In [ ]:
import pandas as pd
train_df = pd.DataFrame(X_train)
train_df['label'] = y_train  # Add the target variable as a column


In [ ]:
# import pycaret classification and init setup
from pycaret.classification import *
s = setup(train_df, target = 'label', session_id = 123)

,Description,Value
0,Session id,123
1,Target,label
2,Target type,Binary
3,Original data shape,"(7004, 101)"
4,Transformed data shape,"(7004, 101)"
5,Transformed train set shape,"(4902, 101)"
6,Transformed test set shape,"(2102, 101)"
7,Numeric features,100
8,Preprocess,True
9,Imputation type,simple


In [ ]:
# import ClassificationExperiment and init the class
from pycaret.classification import ClassificationExperiment
exp = ClassificationExperiment()

In [ ]:
exp.setup(train_df, target = 'label', session_id = 123)



,Description,Value
0,Session id,123
1,Target,label
2,Target type,Binary
3,Original data shape,"(7004, 101)"
4,Transformed data shape,"(7004, 101)"
5,Transformed train set shape,"(4902, 101)"
6,Transformed test set shape,"(2102, 101)"
7,Numeric features,100
8,Preprocess,True
9,Imputation type,simple


In [ ]:
best = compare_models()


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:12:50
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
svm,SVM - Linear Kernel,0.4971,0.4853,0.7457,0.4987,0.5722,-0.0058,-0.0218,0.0490
nb,Naive Bayes,0.4916,0.4781,0.4704,0.4912,0.4803,-0.0167,-0.0167,0.0220
ridge,Ridge Classifier,0.4808,0.4737,0.4802,0.4818,0.4806,-0.0383,-0.0384,0.0220
lda,Linear Discriminant Analysis,0.4804,0.4736,0.4802,0.4813,0.4804,-0.0391,-0.0392,0.0290
lr,Logistic Regression,0.4800,0.4737,0.4782,0.4808,0.4790,-0.0400,-0.0400,0.4910
ada,Ada Boost Classifier,0.4602,0.4405,0.4447,0.4597,0.4519,-0.0796,-0.0796,0.8740
knn,K Neighbors Classifier,0.4164,0.3911,0.4231,0.4174,0.4201,-0.1673,-0.1674,0.0480
gbc,Gradient Boosting Classifier,0.3698,0.3129,0.3721,0.3707,0.3711,-0.2603,-0.2606,4.7610
qda,Quadratic Discriminant Analysis,0.3672,0.3210,0.3708,0.3682,0.3694,-0.2656,-0.2658,0.0350
rf,Random Forest Classifier,0.3003,0.1757,0.2974,0.2985,0.2978,-0.3994,-0.3998,1.3470


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

array([0, 1], dtype=int8)